In [14]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-openai

In [15]:
import os
# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
# os.environ["TAVILY_API_KEY"] = "your-api-key"
# os.environ["OPENAI_API_KEY"] = 'your-api-key'

![](2024-10-25-18-53-39.png)

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

search = TavilySearchResults()

search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1738080958, 'localtime': '2025-01-28 08:15'}, 'current': {'last_updated_epoch': 1738080900, 'last_updated': '2025-01-28 08:15', 'temp_c': 5.6, 'temp_f': 42.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 118, 'wind_dir': 'ESE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 25, 'feelslike_c': 3.9, 'feelslike_f': 38.9, 'windchill_c': 6.2, 'windchill_f': 43.1, 'heatindex_c': 6.6, 'heatindex_f': 43.9, 'dewpoint_c': 6.6, 'dewpoint_f': 43.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.2, 'gust_mph': 8.3, 'gust_kph': 13.4}}"},
 {'url': 'https://weathershogun.com/weather/usa

In [2]:
from langchain_core.tools import tool

@tool 
def create_file(content: str, filename: str):
    """
    Create a file with the given content.
    """
    with open(filename, "w") as f:
        f.write(content)


In [3]:
create_file.invoke({"content": "file content", "filename": "file.txt"}) 

In [4]:
tools = [search, create_file]

In [5]:
# Set up the model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [6]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [8]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [9]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Write a file named pancakes.txt with updated information from the web about why pancakes are the best."})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'why pancakes are the best'}`


[{'url': 'https://cookingpro.net/why-pancakes-are-the-best-breakfast/', 'content': "In this article, we'll explore the myriad reasons why pancakes are the best breakfast choice, diving into their history, nutritional benefits, and creative serving ideas that keep them at the forefront of breakfast choices."}, {'url': 'https://cookingpro.net/why-do-restaurant-pancakes-taste-better/', 'content': "The Science of Pancake Making Before we dive into the reasons why restaurant pancakes taste better, it's essential to understand the science behind pancake making. Pancakes are a type of batter-based food, made from a mixture of flour, sugar, eggs, and milk."}, {'url': 'https://www.americastestkitchen.com/articles/8098-buttermilk-makes-the-best-pancakes', 'content': "But what exactly is it about buttermilk that makes pancakes so good? Flavor is the obvious perk: The cul

{'input': 'Write a file named pancakes.txt with updated information from the web about why pancakes are the best.',
 'output': 'I have created a file named "pancakes.txt" with updated information about why pancakes are considered the best. If you need anything else, feel free to ask!'}

In [22]:
# Adding memory

In [23]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I help you, Bob?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I help you, Bob?'}

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [25]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [26]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Bob! How can I help you, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob! How can I help you, Bob?'}